In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models,callbacks
from sklearn.model_selection import train_test_split
import joblib
from tensorflow.keras.optimizers import Adam
from keras.models import load_model

In [2]:
data=pd.read_csv("dataset.csv")

In [3]:
data.drop(columns=["Angle"],inplace=True)


In [4]:
num_samples=30
k=data.shape[0]//(num_samples)
mat=np.eye(9)
test=np.zeros(shape=(k))
for i in range(9):
  for j in range(k//9):
    test[(k//9)*i+j]=i


In [5]:
tr=np.array(data)
train=np.zeros(shape=(k,num_samples,15))
for i in range(k):
  tmp=np.zeros(shape=(num_samples,15))
  for j in range(num_samples):
    tmp[j]=tr[num_samples*i+j]
  train[i]=tmp
train=train/1024

In [6]:
test_size = 0.2
# test=test.reshape(-1,8,1)
X_train, X_test, y_train, y_test = train_test_split(train, test, test_size=test_size, random_state=42)
X_test.shape
y_test

In [7]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(num_samples, 15, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(9, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
checkpoint_callback = callbacks.ModelCheckpoint('best_weights.h5',
                                                 monitor='val_accuracy',
                                                 save_best_only=True,
                                                 save_weights_only=True,
                                                 mode='max',
                                                 verbose=1)

history = model.fit(X_train, y_train, epochs=50, batch_size=32,
                    validation_data=(X_test, y_test),
                    callbacks=[checkpoint_callback])
model.load_weights('best_weights.h5')


In [8]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)
# Plot training and validation accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.legend()

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()
plt.show()


In [9]:
model.save('my_model.h5')